In [30]:
import pickle
import re
from tqdm import tqdm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances

In the block below we are reading the raw data from articles and tables.

In [2]:
rawArticles = pickle.load(open("NewsDataset/final_article_tables.pkl","rb"))
rawTables = pickle.load(open("NewsDataset/final_dict_tables.pkl","rb"))

In the block next, we are formatting the articles data by removing links, multiples spaces and special characters. Each article has several links. Then we get each one. Finally, we read all text of each link.

In [3]:
formattedArticles = []

for i in tqdm(rawArticles.keys()):
    
    keyArticle = i
     
    dataArticle = rawArticles[keyArticle]

    totalArticles = len(dataArticle)

    for i in range(totalArticles):
    
        fullText = str(dataArticle[i][1])
    
        #removing special characters
        formattedText = re.sub('[^A-Za-z]+',' ',fullText)
    
        #removing left and right spaces
        formattedText = formattedText.lstrip()
        formattedText = formattedText.rstrip()
    
        formattedArticles.append([keyArticle,formattedText])

100%|██████████| 90000/90000 [04:31<00:00, 331.75it/s]


In the next block, we remove empty values from articles.

In [4]:
for article in tqdm(formattedArticles):
    if (article[1] == ''):
        formattedArticles.remove(article)

 98%|█████████▊| 291676/298725 [01:34<00:02, 3090.30it/s]


In the block next, we are reading and formatting the raw table data. Each indice may have multiple tables. So, we collect each table separately. Next, we add tables in the corpus.

In [5]:
formattedTables = []

for i in tqdm(rawTables.keys()):
    
    dataTable = rawTables[i]
    
    totalTalbes = len(dataTable)
    
    for j in range(totalTalbes):
        
        keyTable = dataTable[j][0][0]
        
        keyCaption = dataTable[j][0][1]
        
        tableText = str(dataTable[j][1:])
    
        fullTextTable = re.sub('[^A-Za-z]+',' ',tableText)
        
        #removing left and right spaces
        fullTextTable = fullTextTable.lstrip()
        fullTextTable = fullTextTable.rstrip()
        
        formattedTables.append([keyTable,keyCaption,fullTextTable])

100%|██████████| 90000/90000 [00:57<00:00, 1576.14it/s]


In [6]:
for tables in tqdm(formattedTables):
    if (tables[2] == ''):
        formattedTables.remove(tables)

 97%|█████████▋| 298855/308759 [02:02<00:04, 2430.46it/s]


tf-idf method

In [7]:
corpusTables = []

for tables in formattedTables:
    
    corpusTables.append(tables[2])

In [90]:
temp1 = np.asarray(formattedArticles[0:10])
corpusArticles = temp1[:,1]

In [9]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer='word')
tablesByCharacteristics = vectorizer.fit_transform(corpusTables)

In [91]:
articlesByCharacteristics = vectorizer.transform(corpusArticles)

In [11]:
def getTotalTablesByArticle(idArticle):

    countTopTables = 0

    for tables in formattedTables:
    
        if tables[0] == int(idArticle):
        
             countTopTables = countTopTables + 1

    return countTopTables

In [77]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
    
        #index = distanceVector.index(topkDistance)
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [13]:
def getAccuracy(idRankedTables, idQueryGoal):

    totalRanquedTables = len(idRankedTables)

    match = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            match = match + 1

    accuracy = match / totalRanquedTables
    
    return accuracy

In [93]:
idQueryGoal = int(formattedArticles[0][0])

distanceVector = pairwise_distances(articlesByCharacteristics, tablesByCharacteristics, metric='cosine')

#countTopTables = getTotalTablesByArticle(idQueryGoal)
    
#topK = nsmallest(countTopTables, distanceVector[0])

#idRankedTables = getIdRankedTalbes(topK,distanceVector[0])

In [83]:
print(getAccuracy(idRankedTables,idQueryGoal))

0.0
